In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import json
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt

from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard.writer import SummaryWriter

from sklearn.model_selection import train_test_split

from common import data
from common.training import training_loop, get_predictions

from algos import cbl

from common.models import resnet, deeplab
from common import metrics


# Hyperparameters

In [37]:
log_dir = "./outputs/cbl"

horizon = 1024
stride = 512

batch_size_con = 32
batch_size_sup = 64

num_epochs_con = 10
num_epochs_sup = 5

learning_rate_con = 1e-3
learning_rate_sup = 1e-3

weight_decay_con = 1e-2
weight_decay_sup = 1e-2

supervised_percentage = 0.2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


# Contrastive

## Datasets

In [38]:
paths = data.get_dataset_paths("../data")
train, test = train_test_split(paths, test_size=0.1, random_state=42)


In [39]:
m_data_train = data.Marconi100Dataset(train, scaling=data.Scaling.STANDARD)
m_data_test = data.Marconi100Dataset(test, scaling=data.Scaling.STANDARD)


Loading: 100%|██████████| 25/25 [00:13<00:00,  1.89it/s]


In [40]:
dataset_train = data.UnfoldedDataset(m_data_train, horizon=horizon, stride=stride)
dataset_test = data.UnfoldedDataset(m_data_test, horizon=horizon, stride=stride)


In [41]:
train_loader = DataLoader(
    dataset_train,
    batch_size_con,
    shuffle=True,
)
test_loader = DataLoader(
    dataset_test,
    batch_size_con,
    shuffle=False,
)


## Model

In [42]:
backbone = resnet.ResNetFeatures(
    resnet.Bottleneck, resnet.RESNET50_LAYERS, data.NUM_FEATURES
)
optim = Adam(backbone.parameters(), lr=learning_rate_con, weight_decay=weight_decay_con)

engine_contrastive = cbl.CBLFeatsEngine(
    backbone, device, optim, 0.5, cbl.left_to_right_flipping(), cbl.crop_and_resize()
)


## Training

In [ ]:
with SummaryWriter(log_dir + "/contrastive") as writer:
    training_loop(
        engine=engine_contrastive,
        num_epochs=num_epochs_con,
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        writer=writer,
        save_path=log_dir + "/contrastive/models",
    )


# Supervised

## Dataset

In [ ]:
n_samples = int(len(test) * supervised_percentage)
m_data_sup = data.Marconi100Dataset(train[:n_samples], scaling=data.Scaling.STANDARD)

dataset_sup = data.UnfoldedDataset(m_data_sup, horizon=horizon, stride=stride)
sup_loader = DataLoader(
    dataset_sup,
    batch_size_sup,
    shuffle=True,
)


## Model

In [45]:
for p in backbone.parameters():
    p.requires_grad = False

backbone.nodes = [resnet.LAYER_1, resnet.LAYER_4]

model = deeplab.DeepLabNet(backbone, backbone_channels=[256, 2048], out_feats=1)

optim = Adam(
    model.head.parameters(), lr=learning_rate_sup, weight_decay=weight_decay_sup
)
engine_supervised = cbl.CBLSupervisedEngine(model, device)


In [47]:
batch = next(iter(test_loader))

engine_supervised.val_step(batch)

ValueError: Target size (torch.Size([32, 1024])) must be the same as input size (torch.Size([32, 1024, 1]))

## Training

In [ ]:
with SummaryWriter(log_dir + "/supervised") as writer:
    training_loop(
        engine=engine_supervised,
        num_epochs=num_epochs_sup,
        train_dataloader=sup_loader,
        test_dataloader=test_loader,
        writer=writer,
        save_path=log_dir + "/supervised/models",
    )


# Evaluating

In [ ]:
# Cost model
cmodel = metrics.default_cmodel()


In [ ]:
all_errors, all_labels = get_predictions(engine_supervised.predict, test_loader)


In [ ]:
cmodel.fit(all_errors, all_labels).optimize()


In [ ]:
metrics.plot_cost(cmodel)


In [ ]:
metrics.plot_errors_curve(
    cmodel.false_positives, cmodel.false_negatives, cmodel.thresholds, figsize=(15, 5)
)


In [ ]:
metrics.plot_precision_recall_f1_curve(
    cmodel.precision, cmodel.recall, cmodel.f1_score, cmodel.thresholds
)
